In [1]:
import pandas as pd
import os

from sqlalchemy import create_engine, MetaData, Table
from dotenv import load_dotenv

from datetime import datetime, timedelta
today = datetime.now()


# --- 2. CONSTRUIR LA URL DE LA BASE DE DATOS DE FORMA SEGURA ---
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
yesterday = today - timedelta(days=1)
db_url = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
load_dotenv()
engine = create_engine(db_url)


df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)                    

In [3]:
df_jugadores_fantasy = pd.read_sql("SELECT * FROM valor_jugador_fantasy", engine)

In [4]:
df_jugadores_fantasy

,id_valor,id_jugador,valor_actual,popularidad
0,2386,1,4.49,0.0
1,2387,2,4.27,0.0
2,2388,3,6.32,0.0
3,2389,4,0.65,0.0
4,2390,5,3.47,0.0
...,...,...,...,...
3260,8904,3275,0.00,0.0
3261,8905,3276,0.00,0.0
3262,8906,3277,0.00,0.0
3263,8907,3278,0.00,0.0


In [5]:
df_plantillas_fantasy = pd.read_sql("SELECT * FROM plantilla_fantasy", engine)

In [6]:
df_plantillas_fantasy

,id_plantilla_fantasy,id_equipo_fantasy,id_jugador,precio_compra,es_titular,es_capitan
0,1,3,244,3.16,1,0
1,2,3,663,3.50,1,0
2,3,3,966,3.15,1,0
3,4,3,1117,2.16,1,0
4,5,3,1229,6.52,1,0
...,...,...,...,...,...,...
70,86,10,2167,2.24,1,0
71,87,10,2319,0.84,1,0
72,88,10,2702,0.79,1,0
73,89,10,2786,1.78,0,0


In [19]:
lista = []
for i in range(len(df_plantillas_fantasy)):
 
    lista.append(df_plantillas_fantasy.iloc[i]['id_jugador'])

cantidad_de_plantillas = len(lista) / 15

for i in range(len(lista)):
    veces_que_se_repite = lista.count(lista[i])
    popularidad = (veces_que_se_repite / cantidad_de_plantillas) * 100
    print(f"El jugador {lista[i]} tiene una popularidad de {popularidad}%")
        

El jugador 244.0 tiene una popularidad de 20.0%
El jugador 663.0 tiene una popularidad de 20.0%
El jugador 966.0 tiene una popularidad de 40.0%
El jugador 1117.0 tiene una popularidad de 20.0%
El jugador 1229.0 tiene una popularidad de 20.0%
El jugador 1261.0 tiene una popularidad de 20.0%
El jugador 1276.0 tiene una popularidad de 20.0%
El jugador 1879.0 tiene una popularidad de 20.0%
El jugador 1881.0 tiene una popularidad de 40.0%
El jugador 1987.0 tiene una popularidad de 20.0%
El jugador 2351.0 tiene una popularidad de 20.0%
El jugador 2406.0 tiene una popularidad de 20.0%
El jugador 2710.0 tiene una popularidad de 20.0%
El jugador 2722.0 tiene una popularidad de 20.0%
El jugador 2785.0 tiene una popularidad de 20.0%
El jugador 3.0 tiene una popularidad de 20.0%
El jugador 53.0 tiene una popularidad de 20.0%
El jugador 83.0 tiene una popularidad de 20.0%
El jugador 110.0 tiene una popularidad de 20.0%
El jugador 145.0 tiene una popularidad de 40.0%
El jugador 198.0 tiene una popul